In [22]:
# Import required libraries
import numpy as np
import pandas as pd


In [ ]:
# Read input CSV file
file_path = 'problem.csv'  # Replace with the exact name of the file
data = pd.read_csv(file_path, index_col=0)

# Extract Supply (last column) and Demand (last row)
supply = data['Supply'].iloc[:-1].values
demand = data.loc['Demand', :].values[:-1]

# Extract cost matrix (everything except the last row and column)
cost_matrix = data.iloc[:-1, :-1].values

# Print extracted data
print("Supply (S1, S2, S3):", supply)
print("Demand (D1, D2, D3, D4):", demand)
print("Cost Matrix:")
print(cost_matrix)


Supply (S1, S2, S3): [500. 700. 800.]
Demand (D1, D2, D3, D4): [400. 900. 200. 500.]
Cost Matrix:
[[12 13  4  6]
 [ 6  4 10 11]
 [10  9 12  4]]


In [24]:
def northwest_corner(supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros((len(supply), len(demand)))

    i = j = 0
    while i < len(supply) and j < len(demand):
        allocation[i][j] = min(supply[i], demand[j])
        if supply[i] < demand[j]:
            demand[j] -= supply[i]
            i += 1
        else:
            supply[i] -= demand[j]
            j += 1
    return allocation

# Apply Northwest Corner Rule
nw_allocation = northwest_corner(supply, demand)
print("Northwest Corner Allocation:")
print(nw_allocation)


Northwest Corner Allocation:
[[400. 100.   0.   0.]
 [  0. 700.   0.   0.]
 [  0. 100. 200. 500.]]


In [25]:
def minimum_cost_method(cost_matrix, supply, demand):
    # Copy supply and demand arrays to avoid modifying the originals
    supply = supply.copy()
    demand = demand.copy()

    # Ensure allocation is float for flexibility
    allocation = np.zeros(cost_matrix.shape, dtype=float)

    # Convert cost matrix to float to support np.inf
    costs = cost_matrix.astype(float)

    # Loop until all supply and demand are satisfied
    while np.any(supply > 0) and np.any(demand > 0):
        # Find the index of the minimum cost
        i, j = np.unravel_index(np.argmin(costs, axis=None), costs.shape)

        # Allocate resources
        allocation[i][j] = min(supply[i], demand[j])
        if supply[i] <= demand[j]:
            demand[j] -= supply[i]
            supply[i] = 0
            costs[i, :] = np.inf  # Remove the row (set to infinity)
        else:
            supply[i] -= demand[j]
            demand[j] = 0
            costs[:, j] = np.inf  # Remove the column (set to infinity)

    return allocation

# Apply the Minimum Cost Method
min_cost_allocation = minimum_cost_method(cost_matrix, supply, demand)
print("Minimum Cost Allocation:")
print(min_cost_allocation)


Minimum Cost Allocation:
[[300.   0. 200.   0.]
 [  0. 700.   0.   0.]
 [100. 200.   0. 500.]]


In [26]:
def minimum_row_cost(cost_matrix, supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros(cost_matrix.shape, dtype=float)  # Ensure allocation is float for flexibility

    for i in range(len(supply)):  # Iterate through rows
        while supply[i] > 0:  # While there is still supply
            # Find the column (j) with the minimum cost in row i
            available_demand = np.where(demand > 0, cost_matrix[i, :], np.inf)  # Ignore fully satisfied demands
            j = np.argmin(available_demand)

            # If all demands are satisfied (np.inf everywhere), break the loop
            if np.isinf(available_demand[j]):
                break

            # Allocate as much as possible to minimize cost
            allocation[i][j] = min(supply[i], demand[j])
            if supply[i] <= demand[j]:  # Supply exhausted
                demand[j] -= supply[i]
                supply[i] = 0
            else:  # Demand exhausted
                supply[i] -= demand[j]
                demand[j] = 0

    return allocation

# Apply Minimum Row Cost Method
min_row_allocation = minimum_row_cost(cost_matrix, supply, demand)
print("Minimum Row Cost Allocation:")
print(min_row_allocation)


Minimum Row Cost Allocation:
[[  0.   0. 200. 300.]
 [  0. 700.   0.   0.]
 [400. 200.   0. 200.]]


In [27]:
def vogels_method(cost_matrix, supply, demand):
    # Ensure cost_matrix is of type float to handle np.inf
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros(cost_matrix.shape, dtype=float)
    cost_matrix = cost_matrix.astype(float)  # Convert to float for compatibility with np.inf

    while np.any(supply > 0) and np.any(demand > 0):
        # Calculate penalties
        row_diff = np.sort(cost_matrix, axis=1)[:, :2]
        row_penalty = row_diff[:, 1] - row_diff[:, 0]
        col_diff = np.sort(cost_matrix, axis=0)[:2, :]
        col_penalty = col_diff[1, :] - col_diff[0, :]

        # Handle rows/columns with no available supply/demand
        row_penalty[np.all(cost_matrix == np.inf, axis=1)] = -1
        col_penalty[np.all(cost_matrix == np.inf, axis=0)] = -1

        # Choose the largest penalty
        row_max = np.max(row_penalty)
        col_max = np.max(col_penalty)

        if row_max >= col_max:
            i = np.argmax(row_penalty)
            j = np.argmin(cost_matrix[i, :])
        else:
            j = np.argmax(col_penalty)
            i = np.argmin(cost_matrix[:, j])

        # Allocate the minimum of supply or demand
        allocation[i][j] = min(supply[i], demand[j])
        if supply[i] <= demand[j]:  # Supply exhausted
            demand[j] -= supply[i]
            supply[i] = 0
            cost_matrix[i, :] = np.inf  # Remove the row
        else:  # Demand exhausted
            supply[i] -= demand[j]
            demand[j] = 0
            cost_matrix[:, j] = np.inf  # Remove the column

    return allocation

# Apply Vogel's Method
vogel_allocation = vogels_method(cost_matrix, supply, demand)
print("Vogel's Approximation Allocation:")
print(vogel_allocation)


Vogel's Approximation Allocation:
[[  0.   0. 200. 300.]
 [  0. 700.   0.   0.]
 [400. 200.   0. 200.]]


C:\Users\hayuy\AppData\Local\Temp\ipykernel_1632\835898863.py:13: RuntimeWarning: invalid value encountered in subtract
  col_penalty = col_diff[1, :] - col_diff[0, :]
C:\Users\hayuy\AppData\Local\Temp\ipykernel_1632\835898863.py:11: RuntimeWarning: invalid value encountered in subtract
  row_penalty = row_diff[:, 1] - row_diff[:, 0]


In [28]:
def calculate_total_cost(cost_matrix, allocation):
    return np.sum(cost_matrix * allocation)

methods = {
    "Northwest Corner": nw_allocation,
    "Minimum Cost": min_cost_allocation,
    "Minimum Row Cost": min_row_allocation,
    "Vogel's Method": vogel_allocation
}

for method, allocation in methods.items():
    total_cost = calculate_total_cost(cost_matrix, allocation)
    print(f"{method}: Total Cost = {total_cost}")


Northwest Corner: Total Cost = 14200.0
Minimum Cost: Total Cost = 12000.0
Minimum Row Cost: Total Cost = 12000.0
Vogel's Method: Total Cost = 12000.0
